<h1>Introduction to Reinforcement Learning with TensorFlow</h1>
<p>This tutorial was adapted from various <a href="https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724">examples</a> provided by Arthur Juliani. I would recommend their various blog posts for further reading on Reinforcement Learning.</p>
<p>The following notebook introduces some of the key concepts of Reinforcment Learning with Python and TensorFlow. Please make sure that the following import statements run successfully before continuing:</p>

In [1]:
import tensorflow as tf
import gym
import numpy as np

/home/student/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


<p>Now that we have this taken care of, we can begin  with a brief introduction to one subset of Reinforcement Learning methods: Policy based methods. Policy based methods allow us to make our agent improve its performance by determining which policies for acting produce better results. This is different from more state based methods referred to as Q learning methods. <a href="https://flyyufelix.github.io/2017/10/12/dqn-vs-pg.html">This writeup</a> provides a relatively concise comparision of the two methods, but in short: policy methods seek to act optimally while Q learning methods seek to reach optimal states. Both methods may be used to make an intelligent agent, but for the examples we are working with here, our state space is more continuous than discrete.</p>
<p>Our agent should operate by processing its current <i>observation</i> of the world to choose <i>actions</i> which maximize a <i>reward</i> feedback symbol. The specific world our agent will work in is referred to as the <i>environment</i>, and for this example, we will be working with OpenAI's <a href="https://gym.openai.com/docs/">Cart-Pole</a> environment. Running the following code should set up our example environment.</p>

In [2]:
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


<p>Now that we have our environment set up, we can take a look at what it looks like using the render() function. For this environment, our goal is to train an agent to balance a pivoting pole on a rolling cart. The following code will reset our environment and use an agent which randomly performs actions for 20 episodes, each composed of 100 frames.</p>

<p>Note: if the following code fails, try downgrading the pyglet package using the following command:</p>
<div style="background-color:#300a24"><b><p style="color:white">python3 -m pip install pyglet==1.2.4</p></b></div>

In [3]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

Episode finished after 16 timesteps
Episode finished after 40 timesteps
Episode finished after 30 timesteps
Episode finished after 17 timesteps
Episode finished after 17 timesteps
Episode finished after 10 timesteps
Episode finished after 19 timesteps
Episode finished after 28 timesteps
Episode finished after 14 timesteps
Episode finished after 12 timesteps
Episode finished after 15 timesteps
Episode finished after 28 timesteps
Episode finished after 23 timesteps
Episode finished after 18 timesteps
Episode finished after 26 timesteps
Episode finished after 11 timesteps
Episode finished after 24 timesteps
Episode finished after 60 timesteps
Episode finished after 15 timesteps
Episode finished after 14 timesteps


<h1>Observation Space</h1>
<p>It is important to note what our observation space is. Our observations represent how our agent perceives the world, and in the case of the CartPole environment, our observations consist of 4 numbers representing the position of the cart, the velocity of the cart, the angle of the pole, and the rotational velocity of the pole. Additional information about the CartPole environment can be found <a href="https://github.com/openai/gym/wiki/CartPole-v0">here</a>.</p>

In [4]:
print('Observation space:', env.observation_space)

Observation space: Box(4,)


<h1>Action Space</h1>
<p>The other key aspect our AI needs to understand is the action space, which represents what choices our agent has each timestep. In this example, the cart has only 2 options: push the cart left or push the cart right.</p>

In [5]:
print('Action space:', env.action_space)

Action space: Discrete(2)


<h1>Rewards</h1>
<p>Now that we know what the world looks like and what our options are, we need to understand how we rate our agent's performance. In this example, the agent receives a reward of 1 point for each timestep. The episode will continue to give the agent rewards until either of the following occur:</p>
<ol>
<li>The cart moves too far left or right off screen.</li>
<li>The pole falls over.</li>
<li>The episode reaches 200 timesteps.</li>
</ol>
<p>As a result, we should expect that an intelligent agent will work to stay on screen while keeping the pole balanced upright for as long as it can (up to 200 timesteps). In order to optimally train our agent, we train the agent using to maximize its expected rewards by backpropogating discounted rewards. This means that after each episode, we look back at our experience and add our final reward discounted by our parameter gamma ($\gamma$).</p>
<p>Gamma can be interpretted as how far/near -sighted our agent is. With a value close to 0, our agent does not care as much about future rewards. With a value close to 1, our agent cares more about future rewards. Typically, you would want a value close to, but not necessarily 1. For this example, we choose $\gamma=0.99$.</p>

In [6]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

<p>With these abstract considerations, we can start thinking about how we want our agent to work. Each frame, our agent will receive 4 numerical inputs and produce one of two possible outputs. We can think of our agent as a function of 4 inputs which produces a singular output:</p>
$$f(x_1,x_2,x_3,x_4) \to y$$
<p>We will now start working on implementing our RL agent. We will define a class for our agent which will allow us to define the size of our input (our state information) and the size of our output (our action choice) so that we can more easily adapt this code for later projects. Our constructor begins by defining a set of TensorFlow placeholders for each input. From there, we feed everything into a dense hidden layer of neurons. The neural network architecture for this agent is illustrated below:</p>
<img src="Neural_Network_CartPole.png"></img>

In [22]:
class Agent:
    #Agent constructor to allow us to change the state/action size
    def __init__(self, state_size, action_size, hidden_size=8):
        #Create placeholders for passing in our state information
        self.input = tf.placeholder(shape=[None, state_size], dtype=tf.float32)
        #Connect our input to the first, hidden layer
        self.hidden_layer = tf.layers.dense(self.input, hidden_size, activation=tf.nn.relu)
        #Connect our hidden layer to our output layer for our 2 actions
        self.output = tf.layers.dense(self.hidden_layer, action_size, activation=tf.nn.softmax)
        #Determine which action (0 or 1) is picked by picking the max output node
        self.choice = tf.argmax(self.output, 1)

#These last few lines delete any existing TensorFlow variables and create an instance of our agent.
tf.reset_default_graph()
my_agent = Agent(state_size=4, action_size=2)
#print out the number of trainable variables for our agent (for visualization purposes only)
trainable_vars = 0
for variable in tf.trainable_variables():
    shape = variable.get_shape()
    variable_parameters = 1
    for dimension in shape:
        variable_parameters *= dimension.value
    trainable_vars += variable_parameters
print('Number of trainable variables for our agent:',trainable_vars)

Number of trainable variables for our agent: 58


<p>The above code sets up our agent's neural network like the diagram shown above, but it is missing a key part:<br>
the actual the ability to train the network and change it's variables! We will explore two ways in which we might accomplish this. In total, for our agent with just one hidden layer of 8 neurons, 4 input values, and an output layer for 2 actions, our agent has over 50 unique parameters to train. As the state, action, and hidden layer sizes increase, the number of trainable parameters increases dramatically, so it is important that we are training our agent efficiently.</p>

<p>When many people think of how they can improve the performance of an AI, genetic algorithms often come to mind. This will be the first method we explore. For this method, we will take the best performing agents and tweak their values slightly. For this example, we will take two agents and then make a random selection of half of one agent's variables with those of another as our child agent. This agent will thus be based on both of its two parent agents' parameters. From here, we will mutate a random subset of the child agent's parameters to ensure that our agents do not immediately converge to a suboptimal agent.</p>